In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import UnexpectedAlertPresentException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.firefox.options import Options
import os
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
from selenium.webdriver.common.keys import Keys
import time
import re

In [2]:
rootdir = "C:/Users/minif/OneDrive/Desktop/"
tickers = [
    'T',
    'CTL',
    'S',
    'TMUS',
    'VZ',
    'FTR',
    'TDS',
    'USM'
]
EVs = [[],[],[],[],[],[],[],[]]
dates = []
foundDates = False

In [3]:
browser = webdriver.Firefox()
delay = 10 # seconds
#login
browser.get("https://ycharts.com/companies/GOOG/enterprise_value")

browser.find_element_by_xpath('//a[contains(text(), "Sign In")]').click()

WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.ID, 'id_username')))

username_box = browser.find_element_by_xpath('//*[@id="id_username"]')
password_box = browser.find_element_by_xpath('//*[@id="id_password"]')

username_box.send_keys('jfeitel1@jhu.edu')
password_box.send_keys('aINVgEJmX0H&QrRl')
password_box.send_keys(Keys.ENTER)
WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.XPATH, '//div[@class="dataColLeft"]')))

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="25ac955d-2e7f-4725-8409-4fc5c287cd84", element="aa023111-da13-4af1-a5d5-3128923cabfa")>

In [4]:
def findEV(html, num):
    #print(html)
    startIndex = html.find('ng-show="!initPhase &amp;&amp; !displayDataLoader')
    #print(startIndex)
    while('tr' in html):
        startIndex = html.find('<td', startIndex)
        startIndex = html.find('>', startIndex)
        endIndex = html.find('<', startIndex)
        date = html[startIndex+1: endIndex]
        while('\n' in date):
            date = re.sub('\n', '', date)
        while(' ' in date):
            date = re.sub(' ', '', date)
        if(date == ''):
            break
        if(foundDates == False):
            dates.append(date)
        startIndex = html.find('<td', endIndex)
        startIndex = html.find('>', startIndex)
        endIndex = html.find('<', startIndex)
        EV = html[startIndex+1: endIndex]
        while('\n' in EV):
            EV = re.sub('\n', '', EV)
        while(' ' in EV):
            EV = re.sub(' ', '', EV)
        if('B' in EV):
            EV = re.sub('B', '', EV)
        EVs[num].append(EV)
        html = html[endIndex:]
        startIndex = 0
        endIndex = 0
    
def getEV(ticker, num):
    browser.get('https://ycharts.com/companies/' + ticker + '/enterprise_value')
    WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.XPATH, '//input[@ng-model="startDate"]')))
    startDate = browser.find_element_by_xpath('//input[@ng-model="startDate"]')
    for i in range(10):
        startDate.send_keys(Keys.BACK_SPACE)
    startDate.send_keys('06/29/2007')
    endDate = browser.find_element_by_xpath('//input[@ng-model="endDate"]')
    for i in range(10):
        endDate.send_keys(Keys.BACK_SPACE)
    endDate.send_keys('11/15/2018')
    getDataBtn = browser.find_element_by_xpath('//input[@value="Get Data"]')
    getDataBtn.click()
    html = browser.page_source
    findEV(html, num)
    try:
        while(True):
            # look for the next button and click it, then scrape
            WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.XPATH, '//a[@class="nextBtn"][@ng-click="newPage(\'next\')"]')))
            nextBtn = browser.find_element_by_xpath('//a[@class="nextBtn"][@ng-click="newPage(\'next\')"]')
            nextBtn.click()
            html = browser.page_source
            findEV(html, num)
    except TimeoutException:
        pass

In [5]:
for n in range(len(tickers)):
    getEV(tickers[n], n)
    foundDates = True
browser.close()

In [57]:
def write_data_excel():
    df_output = pd.DataFrame({'Dates': dates, 
                              tickers[0]: EVs[0], 
                              tickers[1]: EVs[1], 
                              tickers[2]: EVs[2], 
                              tickers[3]: EVs[3], 
                              tickers[4]: EVs[4], 
                              tickers[5]: EVs[5],
                              tickers[6]: EVs[6], 
                              tickers[7]: EVs[7]})
    
    writer = pd.ExcelWriter('C:/Users/minif/OneDrive/Desktop/CellularEV.xlsx')
    df_output.to_excel(writer,'Sheet1')
    writer.save()
    
    return df_output

In [7]:
dates2 = dates.copy()

for n in range(len(dates2)):
    if('.' in dates2[n]):
        dates2[n] = dates[n].replace('.', ' ')
    dates2[n] = dates2[n].replace(',', ', ')
    word = list(dates2[n])
    if(word[3].isdigit() == False and word[3] != ' '):
        word[3] = ' '
    if(word[3].isdigit() == True):
        word.insert(3, ' ')
    if(word[4] == ' '):
        word.pop(4)
    dates2[n] = ''.join(word)

dates = dates2

In [62]:
for n in range(len(dates)):
    word = list(dates[n])
    if(word[4] == 'l'):
        del(word[4])
    dates[n] = ''.join(word)
    

In [54]:
EVs[2].append('N/A')

In [55]:
for n in range(len(EVs)):
    print(str(n) + " " + str(len(EVs[n])))

0 2881
1 2881
2 2881
3 2881
4 2881
5 2881
6 2881
7 2881


In [63]:
write_data_excel()

,Dates,T,CTL,S,TMUS,VZ,FTR,TDS,USM
0,"Nov 15, 2018",395.10,56.63,56.78,70.21,356.12,17.56,6.021,5.614
1,"Nov 14, 2018",397.94,56.75,56.90,70.24,355.54,17.56,6.034,5.562
2,"Nov 13, 2018",396.63,56.44,57.15,70.49,354.38,17.58,6.044,5.537
3,"Nov 12, 2018",399.90,56.92,56.66,70.04,354.63,17.59,6.051,5.524
4,"Nov 9, 2018",399.25,56.57,57.07,70.71,353.55,17.58,6.079,5.532
5,"Nov 8, 2018",401.79,58.92,57.35,71.62,352.77,17.59,6.067,5.458
6,"Nov 7, 2018",402.23,59.34,57.84,72.12,350.21,17.60,6.073,5.566
7,"Nov 6, 2018",401.21,58.99,57.11,70.79,348.39,17.72,5.950,5.362
8,"Nov 5, 2018",398.81,58.83,57.39,70.89,348.39,17.70,5.900,5.431
9,"Nov 2, 2018",398.01,58.80,57.39,70.57,345.99,17.70,5.964,5.717
